In [1]:
# 1. Imports and Hyperparameters
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm  # For progress bars

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from torchvision.transforms import Compose, ToTensor, Normalize, Resize

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters (Adjust based on the paper's recommendations)
batch_size = 128
learning_rate = 2e-4
num_epochs = 60
image_size = 256
lambda_L1 = 100  # Weight for L1 loss
lambda_perceptual = 10  # Weight for perceptual loss if used

# Directories
train_dir = '/kaggle/input/person-face-sketches/train'  # Update with your dataset path
val_dir = '/kaggle/input/person-face-sketches/val'
test_dir = '/kaggle/input/person-face-sketches/test'

checkpoint_dir = '/kaggle/working/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# Image saving directory
image_dir = '/kaggle/working/images'
os.makedirs(image_dir, exist_ok=True)



In [2]:
# 2. Dataset Preparation
class SketchToFaceDataset(Dataset):
    """
    Custom Dataset for loading sketch and face image pairs.
    """
    def __init__(self, sketch_dir, face_dir, transform=None):
        self.sketch_dir = sketch_dir
        self.face_dir = face_dir
        self.transform = transform
        self.image_names = os.listdir(self.sketch_dir)

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        sketch_name = self.image_names[idx]
        face_name = sketch_name  # Assuming same filename for sketch and face

        sketch_path = os.path.join(self.sketch_dir, sketch_name)
        face_path = os.path.join(self.face_dir, face_name)

        sketch_image = Image.open(sketch_path).convert('RGB')
        face_image = Image.open(face_path).convert('RGB')

        if self.transform:
            sketch_image = self.transform(sketch_image)
            face_image = self.transform(face_image)

        return {'sketch': sketch_image, 'face': face_image}

# Define transformations
transform = Compose([
    Resize((image_size, image_size)),
    ToTensor(),
    Normalize(mean=[0.5], std=[0.5])
])

# Create datasets and dataloaders
train_dataset = SketchToFaceDataset(
    sketch_dir=os.path.join(train_dir, 'sketches'),
    face_dir=os.path.join(train_dir, 'photos'),
    transform=transform
)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = SketchToFaceDataset(
    sketch_dir=os.path.join(val_dir, 'sketches'),
    face_dir=os.path.join(val_dir, 'photos'),
    transform=transform
)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [3]:
# 3. Model Definitions

# Generator Model (U-Net)
class UNetBlock(nn.Module):
    """
    Defines a single block for the U-Net architecture.
    """
    def __init__(self, in_channels, out_channels, down=True, use_dropout=False):
        super(UNetBlock, self).__init__()
        if down:
            self.block = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU(0.2)
            )
        else:
            self.block = nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU()
            )
            if use_dropout:
                self.block.add_module("Dropout", nn.Dropout(0.5))

    def forward(self, x):
        return self.block(x)

class GeneratorUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(GeneratorUNet, self).__init__()
        self.down1 = UNetBlock(in_channels, 64, down=True, use_dropout=False)  # (b, 64, 128, 128)
        self.down2 = UNetBlock(64, 128, down=True, use_dropout=False)          # (b, 128, 64, 64)
        self.down3 = UNetBlock(128, 256, down=True, use_dropout=False)         # (b, 256, 32, 32)
        self.down4 = UNetBlock(256, 512, down=True, use_dropout=False)         # (b, 512, 16, 16)
        self.down5 = UNetBlock(512, 512, down=True, use_dropout=False)         # (b, 512, 8, 8)
        self.down6 = UNetBlock(512, 512, down=True, use_dropout=False)         # (b, 512, 4, 4)
        self.down7 = UNetBlock(512, 512, down=True, use_dropout=False)         # (b, 512, 2, 2)
        self.down8 = UNetBlock(512, 512, down=True, use_dropout=False)         # (b, 512, 1, 1)

        self.up1 = UNetBlock(512, 512, down=False, use_dropout=True)           # (b, 512, 2, 2)
        self.up2 = UNetBlock(1024, 512, down=False, use_dropout=True)          # (b, 512, 4, 4)
        self.up3 = UNetBlock(1024, 512, down=False, use_dropout=True)          # (b, 512, 8, 8)
        self.up4 = UNetBlock(1024, 512, down=False, use_dropout=False)         # (b, 512, 16, 16)
        self.up5 = UNetBlock(1024, 256, down=False, use_dropout=False)         # (b, 256, 32, 32)
        self.up6 = UNetBlock(512, 128, down=False, use_dropout=False)          # (b, 128, 64, 64)
        self.up7 = UNetBlock(256, 64, down=False, use_dropout=False)           # (b, 64, 128, 128)
        self.up8 = nn.Sequential(
            nn.ConvTranspose2d(128, out_channels, 4, 2, 1),
            nn.Tanh()
        )                                                                      # (b, 3, 256, 256)

    def forward(self, x):
        # Downsampling
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)

        # Upsampling with skip connections
        u1 = self.up1(d8)
        u1 = torch.cat([u1, d7], dim=1)
        u2 = self.up2(u1)
        u2 = torch.cat([u2, d6], dim=1)
        u3 = self.up3(u2)
        u3 = torch.cat([u3, d5], dim=1)
        u4 = self.up4(u3)
        u4 = torch.cat([u4, d4], dim=1)
        u5 = self.up5(u4)
        u5 = torch.cat([u5, d3], dim=1)
        u6 = self.up6(u5)
        u6 = torch.cat([u6, d2], dim=1)
        u7 = self.up7(u6)
        u7 = torch.cat([u7, d1], dim=1)
        output = self.up8(u7)
        return output

# Discriminator Model (PatchGAN)
class Discriminator(nn.Module):
    def __init__(self, in_channels=6):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels, 64, 4, 2, 1),  # (b, 64, 128, 128)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),  # (b, 128, 64, 64)
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),  # (b, 256, 32, 32)
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 1, 1, bias=False),  # (b, 512, 31, 31)
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 1)  # (b, 1, 30, 30)
        )

    def forward(self, x, y):
        # Concatenate input and target images along the channel dimension
        xy = torch.cat([x, y], dim=1)
        return self.model(xy)

# Initialize models
generator = GeneratorUNet().to(device)
discriminator = Discriminator().to(device)


In [4]:
# 4. Loss Functions and Optimizers

# Adversarial loss
criterion_GAN = nn.BCEWithLogitsLoss().to(device)
# L1 loss between generated image and target image
criterion_L1 = nn.L1Loss().to(device)

# Optionally, define perceptual loss using VGG19
use_perceptual_loss = False  # Set to True if you want to use perceptual loss
if use_perceptual_loss:
    from torchvision.models import vgg19
    vgg = vgg19(pretrained=True).features[:35].to(device).eval()
    for param in vgg.parameters():
        param.requires_grad = False

    def compute_perceptual_loss(fake_img, real_img):
        fake_features = vgg(fake_img)
        real_features = vgg(real_img)
        perceptual_loss = criterion_L1(fake_features, real_features)
        return perceptual_loss

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))


In [5]:
def load_checkpoint(generator, discriminator, optimizer_G, optimizer_D):
    checkpoint_path = os.path.join(checkpoint_dir, "latest_checkpoint.pth")
    if os.path.exists(checkpoint_path):
        print(f"Loading checkpoint: {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path)
        generator.load_state_dict(checkpoint['generator_state_dict'])
        discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
        optimizer_G.load_state_dict(checkpoint['optimizer_G_state_dict'])
        optimizer_D.load_state_dict(checkpoint['optimizer_D_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        G_losses = checkpoint['G_losses']
        D_losses = checkpoint['D_losses']
        val_losses = checkpoint['val_losses']
        val_accuracies = checkpoint['val_accuracies']
    else:
        print("No checkpoint found. Starting training from scratch.")
        start_epoch = 0
        G_losses = []
        D_losses = []
        val_losses = []
        val_accuracies = []
    return start_epoch, G_losses, D_losses, val_losses, val_accuracies

# Load checkpoints if any
start_epoch, G_losses, D_losses, val_losses, val_accuracies = load_checkpoint(
    generator, discriminator, optimizer_G, optimizer_D)


No checkpoint found. Starting training from scratch.


In [6]:
# 6. Training Loop with Validation and Checkpointing

for epoch in range(start_epoch, num_epochs):
    generator.train()
    discriminator.train()
    epoch_G_loss = 0
    epoch_D_loss = 0

    for i, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
        # Set model input
        sketches = batch['sketch'].to(device)
        faces = batch['face'].to(device)

        # Adversarial ground truths
        valid = torch.ones((sketches.size(0), 1, 30, 30), requires_grad=False).to(device)
        fake = torch.zeros((sketches.size(0), 1, 30, 30), requires_grad=False).to(device)

        ## ---------------------
        ##  Train Generator
        ## ---------------------
        optimizer_G.zero_grad()

        # Generate a batch of images
        fake_faces = generator(sketches)

        # Adversarial loss
        pred_fake = discriminator(fake_faces, sketches)
        loss_GAN = criterion_GAN(pred_fake, valid)

        # L1 loss
        loss_L1 = criterion_L1(fake_faces, faces) * lambda_L1

        # Total generator loss
        if use_perceptual_loss:
            loss_perceptual = compute_perceptual_loss(fake_faces, faces) * lambda_perceptual
            loss_G = loss_GAN + loss_L1 + loss_perceptual
        else:
            loss_G = loss_GAN + loss_L1

        loss_G.backward()
        optimizer_G.step()

        ## ---------------------
        ##  Train Discriminator
        ## ---------------------
        optimizer_D.zero_grad()

        # Real loss
        pred_real = discriminator(faces, sketches)
        loss_real = criterion_GAN(pred_real, valid)

        # Fake loss
        pred_fake = discriminator(fake_faces.detach(), sketches)
        loss_fake = criterion_GAN(pred_fake, fake)

        # Total discriminator loss
        loss_D = (loss_real + loss_fake) * 0.5

        loss_D.backward()
        optimizer_D.step()

        # Update epoch losses
        epoch_G_loss += loss_G.item()
        epoch_D_loss += loss_D.item()

    # Average losses over epoch
    avg_G_loss = epoch_G_loss / len(train_loader)
    avg_D_loss = epoch_D_loss / len(train_loader)
    G_losses.append(avg_G_loss)
    D_losses.append(avg_D_loss)

    
    # ---------------------
    #  Validation Phase
    # ---------------------
    generator.eval()
    discriminator.eval()
    val_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in val_loader:
            sketches = batch['sketch'].to(device)
            photos = batch['face'].to(device)

            # Define 'valid' and 'fake' tensors for the current batch size
            current_batch_size = sketches.size(0)
            valid = torch.ones((current_batch_size, 1, 30, 30), requires_grad=False).to(device)
            fake = torch.zeros((current_batch_size, 1, 30, 30), requires_grad=False).to(device)
    
            fake_photos = generator(sketches)
    
            # Validation loss
            pred_fake = discriminator(fake_photos, sketches)
            loss_GAN = criterion_GAN(pred_fake, valid)
    
            loss_L1 = criterion_L1(fake_photos, photos) * lambda_L1

            if use_perceptual_loss:
                loss_perceptual = compute_perceptual_loss(fake_photos, photos) * lambda_perceptual
                loss_G = loss_GAN + loss_L1 + loss_perceptual
            else:
                loss_G = loss_GAN + loss_L1
    
            val_loss += loss_G.item()
    
            # Compute accuracy of discriminator on validation set
            pred_real = discriminator(photos, sketches)
            pred_fake = discriminator(fake_photos, sketches)
    
            real_labels = torch.ones_like(pred_real, device=device)
            fake_labels = torch.zeros_like(pred_fake, device=device)
    
            # Combine predictions and labels
            preds = torch.cat([pred_real, pred_fake], dim=0)
            labels = torch.cat([real_labels, fake_labels], dim=0)
    
            # Compute accuracy
            predicted = (torch.sigmoid(preds) > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.numel()


    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    val_accuracy = correct / total
    val_accuracies.append(val_accuracy)

    print(f"Epoch [{epoch}/{num_epochs}] "
          f"Train Loss G: {avg_G_loss:.4f}, D: {avg_D_loss:.4f} "
          f"Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Save sample images every epoch
    with torch.no_grad():
        # Generate images using a fixed batch of validation sketches
        sample_batch = next(iter(val_loader))
        sample_sketches = sample_batch['sketch'].to(device)
        sample_faces = sample_batch['face'].to(device)
        fake_sample_faces = generator(sample_sketches)

        # Save images for visualization
        save_image(fake_sample_faces.data[:25], f"{image_dir}/{epoch}_fake.png", nrow=5, normalize=True)
        save_image(sample_faces.data[:25], f"{image_dir}/{epoch}_real.png", nrow=5, normalize=True)
        save_image(sample_sketches.data[:25], f"{image_dir}/{epoch}_sketch.png", nrow=5, normalize=True)

    checkpoint_path = os.path.join(checkpoint_dir, "latest_checkpoint.pth")
    torch.save({
        'epoch': epoch,
        'generator_state_dict': generator.state_dict(),
        'discriminator_state_dict': discriminator.state_dict(),
        'optimizer_G_state_dict': optimizer_G.state_dict(),
        'optimizer_D_state_dict': optimizer_D.state_dict(),
        'G_losses': G_losses,
        'D_losses': D_losses,
        'val_losses': val_losses,
        'val_accuracies': val_accuracies
    }, checkpoint_path)

# 7. Tracking and Saving Losses

# After training is complete, plot the losses
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses, label="G_loss")
plt.plot(D_losses, label="D_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.savefig('train_loss_plot.png')
plt.show()

plt.figure(figsize=(10,5))
plt.title("Validation Loss During Training")
plt.plot(val_losses, label="Val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.savefig('val_loss_plot.png')
plt.show()

plt.figure(figsize=(10,5))
plt.title("Validation Accuracy During Training")
plt.plot(val_accuracies, label="Val_accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig('val_accuracy_plot.png')
plt.show()



Epoch 0/60:   0%|          | 0/162 [00:00<?, ?it/s]

Epoch 0/60:   1%|          | 1/162 [00:05<15:36,  5.82s/it]

Epoch 0/60:   1%|          | 2/162 [00:10<13:49,  5.19s/it]

Epoch 0/60:   2%|▏         | 3/162 [00:15<13:22,  5.05s/it]

Epoch 0/60:   2%|▏         | 4/162 [00:20<12:59,  4.94s/it]

...

Epoch 2/60:  10%|█         | 17/162 [00:59<08:26,  3.49s/it]

Epoch 2/60:  11%|█         | 18/162 [01:02<08:20,  3.47s/it]

Epoch 2/60:  12%|█▏        | 19/162 [01:06<08:19,  3.49s/it]

Epoch 2/60:  12%|█▏        | 20/162 [01:09<08:13,  3.48s/it]

Epoch 2/60:  13%|█▎        | 21/162 [01:12<08:08,  3.46s/it]